In [1]:
import os
import numpy as np
import plotly.graph_objects as go

import gzip
import pickle as pkl

def loadall_results2(path, n_folds):
    result = []
    with gzip.open(  path ,'rb') as f:
        for i in range(n_folds):

            try:
                data = pkl.load(f)

                result.append( data ) 
            except EOFError:

                break
    
    return np.array(result)

In [2]:
n_folds = 10
horizon = 9999

context = 'MNIST'

direct = './results/'
path = os.path.join(direct, 'case2_{}_{}_{}_{}.pkl.gz'.format(context,horizon,n_folds,'ineural') )
result = loadall_results2(path, 20)
result.shape

(11, 9999)

In [4]:

n_folds = 10
horizon = 9999

game = 'LE'
context = 'MNIST'


colors = [   [250,0,0], [255,128,0], [255,128,0], [255,128,120],  ] 
labels = [     'randneuralcbpside', 'random', 'random2', 'ineural' ] 


fig = go.Figure( )

final_regrets = []

for color, label in list(zip(colors, labels))[::-1]:

    r,g,b = color

    direct = './results/'
    path = os.path.join(direct, 'case2_{}_{}_{}_{}.pkl.gz'.format(context,horizon,n_folds,label) )
    result = loadall_results2(path, n_folds)
    result = result[1:]
    result = result.astype(np.float32)

    final_regrets.append( result[:,-1] )
    
    regret =  np.mean(result,0) 
    xcoords = np.arange(0,horizon,1).tolist()
    std =  np.std( result , 0 )
    upper_regret = regret + std


    fig.add_trace(go.Scatter(x=xcoords, y=regret, line=dict(color='rgb({},{},{})'.format(r,g,b),  dash=  'solid' ), mode='lines',  name=label,  showlegend=False )) # 
    fig.add_trace(   go.Scatter( x=xcoords+xcoords[::-1], y=upper_regret.tolist()+regret.tolist()[::-1],  fill='toself', fillcolor='rgba({}, {}, {},0.2)'.format(r,g,b), 
                            line=dict(color='rgba(255,255,255,0)'),   hoverinfo="skip",  showlegend=False )   )


    
fig.update_layout(autosize=False,
                  xaxis_title="Sequence", yaxis_title="Regret and Standard Dev.",  margin=go.layout.Margin( l=0,   r=0,   b=0,    t=0, ),   font=dict(size=20,) )
                  #legend= dict(yanchor="top",y=0.98,xanchor="left",x=0.1

# fig.update_yaxes( type="log" )
# fig.write_image("./hard_LE.pdf")

# fig.update_xaxes( type="log" )
# fig.write_image("./hard_LE_log.pdf")

fig.update_layout( yaxis=dict(range=[0, 10000] ) )


fig.show()
#fig.write_image("./cr_{}_{}.pdf".format(task, game) )

# final_regrets = np.array(final_regrets)
# final = [ ( np.argmin(final_regrets[:,i]), i) for i in range(n_folds) ]